# Úkol č. 2 - předzpracování dat a binární klasifikace (do 2. listopadu 23:59)

  * V rámci tohoto úkolu se musíte vypořádat s příznaky, které jsou různých typů.
  * Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na podmnožiny vhodné k trénování modelu.
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.
  * Ukázka prvních řádků souboru *results.csv*:
  
```
ID,survived
1000,0
1001,1
...
```

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí křížové validace. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor s predikcemi pro vyhodnocovací data (`results.csv`).
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [462]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

from sklearn import metrics, datasets
from sklearn.model_selection import ParameterGrid, train_test_split, KFold, LeaveOneOut
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import KNNImputer

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

Start with dropping of columns:

ID, represents the order of person

ticket, shows structure in which the tickets were sold, does not show the quality of person

cabin - too many missing values (more than 3/4 are missing), too variable as well

home.dest - does not represent any quality and has a lot of missing values

In [463]:
df = pd.read_csv('data.csv')
df_eval = pd.read_csv('evaluation.csv')
dflist = [df, df_eval]
for d in dflist:
    d.drop(columns=['ID', 'ticket', 'cabin', 'home.dest'], inplace=True)

Bruteforce cleaning of one row in fare in evaluation set

In [464]:
median = df_eval[df_eval['pclass'] == 3].fare.median()
for index, row in df_eval.iterrows():
    if np.isnan(row['fare']):
        df_eval.at[index, 'fare'] = median

Add auxiliary column - represents the length of name

In [465]:
for d in dflist:
    name_l = []
    for i in d['name']:
        name_l.append(len(i))
    d['name_length'] = name_l

Extract rank/title of person from names - people with higher ranks have priority.
rank is method of dataframe, use ranking

In [466]:
for d in dflist:
    d['ranking'] = d.name.str.extract(' ([A-Za-z]+)\.', expand=False)
    d['ranking'] = d['ranking'].replace(['Countess', 'Master', 'Jonkheer','Rev', 'Don', 'Dona', 'Lady', 'Sir', 'Dr'], 'noble')
    d['ranking'] = d['ranking'].replace(['Major','Col', 'Capt'], 'army')
    d['ranking'] = d['ranking'].replace('Ms', 'Mrs')
    d['ranking'] = d['ranking'].replace('Mlle', 'Mrs')
    d['ranking'] = d['ranking'].replace('Mme', 'Mrs')
    # lower number is lower priority of saving 
    order = {"army": 1, "Mr": 2,  "Mrs": 3, "noble": 4}
    d['ranking'] = d['ranking'].map(order)
    d['ranking'] = d['ranking'].fillna(0)
    d.drop(columns=['name'], inplace=True)

Create one-hot encoding for categoric values

In [467]:
columns_to_category = ['sex', 'embarked']
df = pd.get_dummies(df, columns=columns_to_category) 
df_eval = pd.get_dummies(df_eval, columns=columns_to_category)

Age is the only column with missing values

In [468]:
for d in dflist:
    display(d.isna().sum(axis=0))

survived         0
pclass           0
sex              0
age            203
sibsp            0
parch            0
fare             0
embarked         2
name_length      0
ranking          0
dtype: int64

pclass          0
sex             0
age            60
sibsp           0
parch           0
fare            0
embarked        0
name_length     0
ranking         0
dtype: int64

### Split data for adding missing values

In [469]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import time
X = df.iloc[:,1:]
y = df.iloc[:,0]
rd_seed = 934
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=rd_seed)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.2, random_state=rd_seed)

### BONUS 2:
Use kNN for filling missing values to 'age' column.

In [470]:
from sklearn.model_selection import ParameterGrid
param_grid = {
    'n_neighbors': range(1,12), 
    'weights': ['uniform', 'distance'],
    'p': range(1,3),
    'n_jobs': [-1],
    'algorithm': ['ball_tree', 'kd_tree', 'brute']
}
param_comb = ParameterGrid(param_grid)


I only use Xtrain set for training the prediction of missing age -  the other sets Xval, Xtest are not affected by this prediction.

In [471]:
err = []
Xnotnull = Xtrain.dropna().copy(deep=True)
Xvalnotnull = Xval.dropna().copy(deep=True)
XtrainKNN = Xnotnull.drop(columns=['age'])
ytrainKNN = Xnotnull.age
for param in param_comb:
    XtrainKNN = Xnotnull.drop(columns=['age'])
    ytrainKNN = Xnotnull.age
    XvalKNN = Xvalnotnull.drop(columns=['age'])
    yvalKNN = Xvalnotnull.age

    scaler = MinMaxScaler()
    XtrainKNN = pd.DataFrame(scaler.fit_transform(XtrainKNN), index=XtrainKNN.index, columns=XtrainKNN.columns)
    XvalKNN = pd.DataFrame(scaler.transform(XvalKNN), index=XvalKNN.index, columns=XvalKNN.columns)

    model = KNeighborsRegressor(**param)
    model.fit(XtrainKNN, ytrainKNN)
  
    err.append(np.sqrt(metrics.mean_squared_error(yvalKNN, model.predict(XvalKNN))))
best_index = np.argmin(err)
best_param_missing = param_comb[best_index]
print("Error in predicting missing age: ", err[best_index])
missing_model = KNeighborsRegressor(**best_param_missing)
missing_model.fit(XtrainKNN, ytrainKNN)

Error in predicting missing age:  11.526424622962903


KNeighborsRegressor(algorithm='brute', n_jobs=-1, n_neighbors=8)

Found the best parameters for predicting missing values. Generate missing values for train and validate dataset.

In [472]:
# predict
X = Xtrain[Xtrain.age.isna()].drop(columns='age')
Xv = Xval[Xval.age.isna()].drop(columns='age')
predicted = missing_model.predict(X)
predictedv = missing_model.predict(Xv)
cnt = 0
for index, row in Xtrain.iterrows():
    if np.isnan(row['age']):
        Xtrain.at[index, 'age'] = predicted[cnt]
        cnt += 1
cnt = 0
for index, row in Xval.iterrows():
    if np.isnan(row['age']):
        Xval.at[index, 'age'] = predictedv[cnt]
        cnt += 1
# no missing data
display(Xtrain.notnull().sum(axis=0))
display(Xval.notnull().sum(axis=0))

pclass         640
age            640
sibsp          640
parch          640
fare           640
name_length    640
ranking        640
sex_female     640
sex_male       640
embarked_C     640
embarked_Q     640
embarked_S     640
dtype: int64

pclass         160
age            160
sibsp          160
parch          160
fare           160
name_length    160
ranking        160
sex_female     160
sex_male       160
embarked_C     160
embarked_Q     160
embarked_S     160
dtype: int64

# TRAINING

In [473]:
from sklearn.tree import DecisionTreeClassifier

In [474]:
from sklearn.model_selection import ParameterGrid
param_grid_train = {
    'max_depth': range(1,25), 
    'criterion': ['entropy', 'gini'],
    'splitter': ['best', 'random'],
    'min_samples_leaf': range(2,5)
}
param_comb_train = ParameterGrid(param_grid_train)

In [475]:
succ = []
for param in param_comb_train:
    model = DecisionTreeClassifier(**param)
    model.fit(Xtrain, ytrain)
    succ.append(metrics.accuracy_score(yval, model.predict(Xval)))

In [476]:
best_param = param_comb_train[np.argmax(succ)]
print("Training success:", np.max(succ))
print("params: ", best_param)
best_model = DecisionTreeClassifier(**best_param)
# merge train and validation set, fit best paramater model
Xtv = pd.concat([Xtrain, Xval])
ytv = pd.concat([ytrain, yval])
best_model.fit(Xtv, ytv)

Training success: 0.8125
params:  {'splitter': 'random', 'min_samples_leaf': 4, 'max_depth': 11, 'criterion': 'entropy'}


DecisionTreeClassifier(criterion='entropy', max_depth=11, min_samples_leaf=4,
                       splitter='random')

# TESTING

In [477]:
X = df.iloc[:,1:]
y = df.iloc[:,0]
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=rd_seed)

Fit model for generating missing values on Train+validation set, generate missing for evaluation and testing set.
Xtrain is now Xtrain + Xvalidation

In [478]:
# fit missing model on bigger set - Train + validation
missing_model = KNeighborsRegressor(**best_param_missing)
X = Xtrain.copy(deep=True).dropna()
# transform
X = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)
# fit
missing_model.fit(X.drop(columns='age'), X.age)
# predict missing values for testing and evaluation
Xt = Xtest[Xtest.age.isna()]
Xt = pd.DataFrame(scaler.transform(Xt), index=Xt.index, columns=Xt.columns)
Xeval = df_eval[df_eval.age.isna()]
Xeval = pd.DataFrame(scaler.transform(Xeval), index=Xeval.index, columns=Xeval.columns)

predictedTest = missing_model.predict(Xt.drop(columns='age'))
predictedEval = missing_model.predict(Xeval.drop(columns='age'))
cnt = 0
for index, row in Xtest.iterrows():
    if np.isnan(row['age']):
        Xtest.at[index, 'age'] = predictedTest[cnt]
        cnt += 1
cnt = 0
for index, row in df_eval.iterrows():
    if np.isnan(row['age']):
        df_eval.at[index, 'age'] = predictedEval[cnt]
        cnt += 1

# no missing data
display(Xtest.notnull().sum(axis=0))
display(df_eval.notnull().sum(axis=0))

pclass         200
age            200
sibsp          200
parch          200
fare           200
name_length    200
ranking        200
sex_female     200
sex_male       200
embarked_C     200
embarked_Q     200
embarked_S     200
dtype: int64

pclass         309
age            309
sibsp          309
parch          309
fare           309
name_length    309
ranking        309
sex_female     309
sex_male       309
embarked_C     309
embarked_Q     309
embarked_S     309
dtype: int64

In [479]:
print("Testing accuracy:", metrics.accuracy_score(ytest, best_model.predict(Xtest)))

Testing accuracy: 0.8


# EVALUATION

In [480]:
df_out = pd.DataFrame()
df_out.insert(0,'survived', best_model.predict(df_eval))
df_out.insert(0,'ID',range(1000,1309))
df_out.to_csv('results.csv',index=False)